In [1]:

# 预处理

import json
from openai import APIConnectionError, OpenAI

import yaml

def load_config(file_path):
    with open(file_path, 'r') as config_file:
        config = yaml.safe_load(config_file)
    return config

config = load_config('config.yaml')

client = OpenAI(
    api_key=config['api']['key']
)

def makecomversation(prompt, usercontent, return_format, temperature=0):
    def sendquestion(prompt, usercontent):
        message = [
                {
                    "role": "system",
                    "content": prompt
                },
                {
                    "role": "user",  
                    "content": usercontent + " Return in JSON format: " + return_format
                }
            ]
        
        print(message)
        chat_completion = client.chat.completions.create(
            model="gpt-4-1106-preview",
            messages=message,
            temperature=temperature,
            response_format={"type": "json_object"}
            )
        return chat_completion

    try:
       chat_completion = sendquestion(prompt, usercontent)
    except Exception as e: 
       chat_completion = sendquestion(prompt, usercontent)
       
    answer = chat_completion.choices[0].message.content
    return json.loads(answer) 

# 取消别名
def remove_aliases(sql):
    prompt = """You are a SQL parsing expert. You need to remove the aliases in sql statement, then return the SQL statement without the aliases. Here is one examples:
    WITH clause is not considered in this task.
    before:
          "SELECT DISTINCT count(anon_1.c_4200) OVER (PARTITION BY anon_1.description) AS c_4528, 
          anon_1.c_141, 
          anon_1.description 
          FROM (
            SELECT film_text.description AS description, 
            count(distinct(film_text.film_id)) AS c_141, 
            count(film_text.title) OVER (ORDER BY film_text.title) AS c_4200 
            FROM film_text 
            WHERE (film_text.film_id >= %(film_id_1)s) 
            GROUP BY film_text.title, film_text.description, film_text.film_id) AS anon_1
    after: 
          SELECT DISTINCT COUNT(film_text.title) OVER (PARTITION BY film_text.description), 
                COUNT(DISTINCT film_text.film_id) 
                film_text.description
          FROM (
              SELECT film_text.description, 
                    COUNT(DISTINCT film_text.film_id) , 
                    COUNT(film_text.title) OVER (ORDER BY film_text.title)
              FROM film_text
              WHERE film_text.film_id >= %(film_id_1)s
              GROUP BY film_text.title, film_text.description, film_text.film_id
          ) ;
    """

    usercontent = f""" You need to remove the aliases from the SQL provided below, then return the SQL statement without the aliases.
    Make sure you return a valid SQL statement, do not generate any other content, it you find problem in sql provided, ignore it.  

    The SQL statement is: 
    {sql}
    """

    return_format = '{"sql":"SQL statement without the aliases"}'

    result = makecomversation(prompt, usercontent, return_format)

    print('remove_aliases  ',result)
  
    return result['sql']

# 添加表名
def place_tablename(sql):
    prompt = """You are a SQL parsing expert. You need to add the table name to all the indicator that appear in the sql statement. """

    usercontent = f""" 
        You need to add the table name to all the indicator that appear in the sql statement。
        Make sure there are no omissions. 
        Do not abbreviate the table name. 
        if the table is is already correctly formatted return the original SQL statement.

        The SQL statement is: 
        {sql}
        """
    
    return_format = '{"sql":"SQL statement with the table name added to all the indicator"}'

    result = makecomversation(prompt, usercontent, return_format)

    print("place_tablename  ",result)

    return result['sql']

# 替换运算符
def replace_logical_operators(sql):
    WHERE_OPS = ('not', 'between', '=', '>', '<', '>=', '<=', '!=', 'in', 'like', 'is', 'exists')
    WHERE_OPS_NL = ('not', 'between', 'equal to', 'more than', 'less than', 'no less than', 'no more than', 'not equal to', 'in', 'like', 'is', 'exists')


    for op, nl in zip(WHERE_OPS, WHERE_OPS_NL):
                            op_str = f" {op} "
                            if  op_str in sql:
                                sql = sql.replace(op_str, f" {nl} ")
                            
    return sql


def preprocess(sql):
    sql = remove_aliases(sql)  
    sql = place_tablename(sql)
    sql = replace_logical_operators(sql)
    return sql



# robust_test

# with open('./data/3_21/home_credit_default.json', 'r') as f:
#     salika = json.load(f)[:10]


# salika_clauses = []
# for item in salika:
#     sql = item['query']
#     clause = preprocess(sql)
#     salika_clauses.append(clause)

sql = """SELECT employees.birth_time, 
                    salaries.to_time 
    FROM employees AS employees JOIN salaries AS salaries ON employees.emp_no = salaries.emp_no 
    WHERE salaries.emp_no in 
            (SELECT dept_emp.emp_no 
                FROM dept_emp AS dept_emp 
                WHERE dept_emp.dept_no = 'd003') 
                and salaries.salary > 90623"""


[{'role': 'system', 'content': 'You are a SQL parsing expert. You need to remove the aliases in sql statement, then return the SQL statement without the aliases. Here is one examples:\n    WITH clause is not considered in this task.\n    before:\n          "SELECT DISTINCT count(anon_1.c_4200) OVER (PARTITION BY anon_1.description) AS c_4528, \n          anon_1.c_141, \n          anon_1.description \n          FROM (\n            SELECT film_text.description AS description, \n            count(distinct(film_text.film_id)) AS c_141, \n            count(film_text.title) OVER (ORDER BY film_text.title) AS c_4200 \n            FROM film_text \n            WHERE (film_text.film_id >= %(film_id_1)s) \n            GROUP BY film_text.title, film_text.description, film_text.film_id) AS anon_1\n    after: \n          SELECT DISTINCT COUNT(film_text.title) OVER (PARTITION BY film_text.description), \n                COUNT(DISTINCT film_text.film_id) \n                film_text.description\n      

In [2]:
## tokenizer 子句分割方法

import re
import sqlparse
from sqlparse.sql import IdentifierList, Identifier, Where
import sqlparse.tokens as Token

def is_subselect(parsed):
    if not parsed.is_group:
        return False
    for item in parsed.tokens:
        if item.ttype is Token.DML and item.value.upper() == 'SELECT':
            return True
    return False

class SQLNode:
    def __init__(self, node_type, value=None, sub_list=None):
        self.type = node_type
        self.value = value
        self.sub_list = sub_list or []

    def __repr__(self):
        return f"node(tt={self.type}, vv={self.value}, ss={self.sub_list})"
    

    def has_sublist(self):
        return len(self.sub_list) > 0
    
    def get_clause(self, depth):
        return str(str(depth)+' '+self.type+' '+self.value)
    
    def merge_subsql(self, depth=1):
        clause_list = []
        for node in self.sub_list:
            if node.has_sublist(): # 有下级管理下级
                clause_list += node.merge_subsql(depth+1)
            
            # 此时node有子节点，而子节点没有子节点，即只有一层嵌套的情况
            clause_list += node.merge_select(depth)
            clause_list += node.merge_from(depth)
            clause_list += node.merge_where(depth)
            # 没有下级则等待管理


            if depth == 1: # 最高级管理，1级没有上级，在这里处理
                clause_list.append(node.get_clause(depth))
        return clause_list
    
    # 下面的merge方法只考虑有一层嵌套的情况：

    def merge_select(self, depth):
        clause_list = []
        if ('SELECT' in self.type ) and (len(self.sub_list) > 0):
            clause_list = [subnode.get_clause(depth) for subnode in self.sub_list]
        return clause_list
        

    def merge_from(self, depth):
        clause_list = []
        if ('FROM' in self.type) and (len(self.sub_list) > 0):
            clause_list = [subnode.get_clause(depth+1) for subnode in self.sub_list if not any(item in subnode.type for item in ['SELECT', 'FROM', 'ORDER BY'])]
            self.sub_list = []
        return clause_list
                    

    def merge_where(self, depth):
        clause_list = []
        if ('WHERE' in self.type) and (len(self.sub_list) > 0):
            # 把subselect1, subselect2替换成相应select的内容
            mapping_table = {
            }
            for item in self.sub_list:
                if 'SELECT' in item.type:
                    mapping_table[item.type] = item.value
                    # remove item from sub_list
                    self.sub_list.remove(item)
                    
            # 替换函数
            def replace_subselect(match):
                subselect_key = match.group(0)  # 获取整个匹配的子字符串，例如：subselect1
                subselect_number = match.group(1)  # 获取匹配的数字部分，例如：1
                mapping_key = f'SELECT{subselect_number}'
                return mapping_table.get(mapping_key)

            # 正则表达式匹配subselect后的数字
            pattern = re.compile(r'subselect(\d+)')

            # 使用sub()函数进行替换
            self.value = pattern.sub(replace_subselect, self.value)

            # self.value =  self.value + ' ' + self.sub_list[0].value
            # self.sub_list.pop(0)

            clause_list = [subnode.get_clause(depth+1) for subnode in self.sub_list]
            self.sub_list = []
        return clause_list

## 子句粗分解    
def decompose(sql):
    parsed = sqlparse.parse(sql)
    stmt = parsed[0]

    # 粗分解
    result,_,_ = split_by_keywords(stmt)
    return result


def split_by_keywords(tokenlist, subselect_num = 0): # subselect_num子查询数量
    # 每一递归处理同一层的token
    # 同一层内可能包含并行嵌套，例如SELECT a, (SELECT b FROM c), (SELECT e FROM f) FROM d，因此需要对subselect编号

    KEYWORDS = {'SELECT', 'FROM', 'WHERE', 'GROUP BY', 'HAVING', 'ORDER BY', 'LIMIT', 'WITH', 'UNION', 'INTERSECT', 'EXCEPT'}
    
    keyword=None # 本层的关键字，当本层不是有效sql层时，keyword始终为None
    clauses='' # 本层的关键字后面的子句
    subsqllist=[] # 本层的子查询
    nodelist = [] # 本层的节点集合

    for token in tokenlist:
        # 为什么采用尾递归的形式：
        # 1. 因为一个token 首先是一个group，其次才会是一个子查询。我们重点考虑的是子查询嵌套的情况

        if is_subselect(token):  
            # 当前的token 是一个子查询，下面的部分需要提取该子查询的Node集合
            s1 = token
            subselect_num += 1
            subsql, _, _ = split_by_keywords(s1.tokens, subselect_num)  
            # 对于同一层的子查询， 不需要区分，所以可以全部归纳到subsqllist中：
            subsqllist += subsql 
            clauses += 'subselect'+str(subselect_num) # 用subselect代替子查询的内容
      
        elif isinstance(token, Where):
            # 因为sqlsparse的实现，WHERE相比其他关键字需要特别处理
            # 这时候的token是一整个where语句

            # 返回的subsql事实上是一个只包含where节点的列表
            subsql, res_clause, _ = split_by_keywords(token.tokens, subselect_num)
            where_node = subsql[0]
            nodelist.append(where_node)

        elif token.value.upper() in KEYWORDS:
            # 当前的Token 是一个关键字
            # 除了第一次遇见关键字即第一个SELECT之外，或者With，其他时刻都不可能是None
            # 第一次遇见关键字，不需要创建新的节点
            if keyword is not None: 
                
                nodelist.append(SQLNode(keyword, clauses, subsqllist))
                # 添加完节点后，初始化子查询列表和子句内容   
                clauses = ''
                subsqllist = []

            # 遇到关键字总要更新关键字
            keyword = token.value.upper()+str(subselect_num)

        elif not token.is_group:
            # 当前的token是一个词
            clauses += token.value

        else : 
            # token是一个group(token组)，它可能包含子查询(subsql)，以及其他词（res_clause）
            subsql, res_clause, _ = split_by_keywords(token.tokens, subselect_num)
            subsqllist += subsql
            clauses += res_clause


    if keyword is not None: # 处理最后一个关键字，如果keyword is None说明本层不是一个有效sql层，可能只有两个indentifier，例如a,b
        nodelist.append(SQLNode(keyword, clauses, subsqllist))
        clauses = ''
        subsqllist = []
    
    # 如果该层是有效sql层（keyword is not None）：
        # 一定会有一个nodelist，而subsqllist为空
    # 如果该层不是有效层，例如只有两个indentifier，那么nodelist为空，而subsqllist不为空

    return nodelist+subsqllist, clauses, subselect_num
    
def get_clause(sql):
    root_node = SQLNode('ROOT', None, decompose(sql))
    clause_list = root_node.merge_subsql()
    return clause_list


In [3]:
# 后处理

# ，删除所有from
def remove_from(clause_list):
    processed_clauses = []
    for clause in clause_list:
        if 'FROM' not in clause:
            processed_clauses.append(clause)
    return processed_clauses

# 删除所有换行符
def remove_linefeed(clause_list):
    return [clause.replace('\n', '') for clause in clause_list]

# 按照数字排序
def sort_caluse(clause_list):
    def custom_sort(query):
        return int(query.split()[0])
    return  sorted(clause_list, key=custom_sort)

# (搁置) 提取所有的标识符
def extract_identifiers(clause_list):
    for clause in clause_list:
        matches = re.findall(r'\b\w+\.\w+\b', clause)
        matches = list(set(matches))  
    return  matches


def remove_with(clause_list):
    processed_clauses = []
    for clause in clause_list:
        if 'WITH' not in clause:
            processed_clauses.append(clause)
    return processed_clauses

def merge_having(clause_list):
    processed_clauses = []
    havings = []
    for clause in reversed(clause_list):
        if 'HAVING' in clause:
            havings.append(clause)
        elif 'GROUP BY' in clause:
            # 如果有GROUP BY，那么HAVING一定在GROUP BY之前
            if len(havings) > 0:
                by = (clause.split()[2])
                index_by = re.findall(r'[a-zA-Z]+(\d+)', by)[0]
                for having in havings:
                    index_having = re.findall(r'[a-zA-Z]+(\d+)', having.split()[1])[0]
                    if index_having == index_by:
                        processed_clauses.append(clause + clean_clause(having))
                        havings.remove(having)
            else:     
                processed_clauses.append(clause)
        else:
            processed_clauses.append(clause)
    return list(reversed(processed_clauses))

def clean_clause(clause):
    cleaned_clause = re.sub(r'^\d+\s*([\w\s]+?)\d+', r'\1', clause)
    return cleaned_clause.strip()

def clean_clause_list(clause_list):
    return [clean_clause(clause) for clause in clause_list]

def after_process(clause_list):
    clause_list = remove_from(clause_list)
    clause_list = remove_linefeed(clause_list)
    clause_list = sort_caluse(clause_list)
    clause_list = remove_with(clause_list)
    clause_list = merge_having(clause_list)
    clause_list = clean_clause_list(clause_list)
    return clause_list

# decomposed_salika_clauses = []
# for sql in salika_clauses:
#     clause_list = get_clause(sql)
#     decomposed_salika_clauses.append(after_process(clause_list))



In [7]:
decomposed_salika_clauses

[['SELECT  credit_card_balance.AMT_RECEIVABLE_PRINCIPAL',
  'WHERE  credit_card_balance.SK_ID_PREV NOT IN ( previous.SK_ID_PREV'],
 ['SELECT  installments_payments.SK_ID_PREV',
  'WHERE  installments_payments.SK_ID_CURR NOT IN ( previous.SK_ID_CURR'],
 ['SELECT  bureau_balance.STATUS',
  'WHERE  bureau_balance.SK_ID_BUREAU NOT IN ( bureau.SK_ID_BUREAU'],
 ['SELECT  pos_cash_balance.SK_DPD',
  'WHERE  pos_cash_balance.SK_ID_CURR NOT IN ( previous.SK_ID_CURR'],
 ['SELECT  bureau.CREDIT_DAY_OVERDUE',
  'WHERE  bureau.SK_ID_BUREAU NOT IN ( bureau_balance.SK_ID_BUREAU'],
 ['SELECT  installments_payments.NUM_INSTALMENT_NUMBER',
  'WHERE  installments_payments.SK_ID_PREV NOT IN ( previous.SK_ID_PREV'],
 ['SELECT  application_test.NAME_HOUSING_TYPE',
  'WHERE  application_test.AMT_ANNUITY NOT IN ( bureau.AMT_ANNUITY'],
 ['SELECT  COUNT(pos_cash_balance.CNT_INSTALMENT_FUTURE)',
  'WHERE  pos_cash_balance.SK_ID_PREV NOT IN ( previous.SK_ID_PREV'],
 ['SELECT  COUNT(pos_cash_balance.SK_DPD_DEF)',


In [11]:
import pandas as pd
des_df = pd.read_csv('HomeCredit_columns_description.csv',index_col=0)
des_df

,Table,Row,Description,Special,Unnamed: 5,Unnamed: 6
1,application_{train|test},SK_ID_CURR,ID of loan in our sample,NaN,NaN,NaN
2,application_{train|test},TARGET,Target variable (1 - client with payment diffi...,NaN,NaN,NaN
5,application_{train|test},NAME_CONTRACT_TYPE,Identification if loan is cash or revolving,NaN,NaN,NaN
6,application_{train|test},CODE_GENDER,Gender of the client,NaN,NaN,NaN
7,application_{train|test},FLAG_OWN_CAR,Flag if the client owns a car,NaN,NaN,NaN
...,...,...,...,...,...,...
120,application_test,AMT_REQ_CREDIT_BUREAU_DAY,Number of enquiries to Credit Bureau about the...,NaN,NaN,NaN
121,application_test,AMT_REQ_CREDIT_BUREAU_WEEK,Number of enquiries to Credit Bureau about the...,NaN,NaN,NaN
122,application_test,AMT_REQ_CREDIT_BUREAU_MON,Number of enquiries to Credit Bureau about the...,NaN,NaN,NaN
123,application_test,AMT_REQ_CREDIT_BUREAU_QRT,Number of enquiries to Credit Bureau about the...,NaN,NaN,NaN


In [ ]:
import pandas as pd
des_df = pd.read_csv('HomeCredit_columns_description.csv',index_col=0)
des_df

In [ ]:
import re
pattern = r'\b\w+\.\w+\b'
indicator_list = []
for sql in decomposed_salika_clauses:
  sql_indicators = []
  for clause in sql:
    matches = re.findall(pattern, clause)
    sql_indicators += matches
  indicator_list.append(sql_indicators)
indicator_list



In [21]:


def get_description(indicators):
  descriptions = []
  for indicator in indicators:
    for index, row in des_df.iterrows():
      if indicator in row['Table']+'.'+row['Row']:
        descriptions.append({indicator:row['Description']})
  return descriptions

descriptions_list = []
for indicators in indicator_list:
  descriptions = get_description(indicators)
  if len(descriptions) != 0:
    descriptions_list.append(descriptions)
descriptions_list

[[{'credit_card_balance.AMT_RECEIVABLE_PRINCIPAL': 'Amount receivable for principal on the previous credit'},
  {'credit_card_balance.SK_ID_PREV': 'ID of previous credit in Home credit related to loan in our sample. (One loan in our sample can have 0,1,2 or more previous loans in Home Credit)'}],
 [{'installments_payments.SK_ID_CURR': 'ID of loan in our sample'}],
 [{'bureau_balance.STATUS': 'Status of Credit Bureau loan during the month (active, closed, DPD0-30,?[C means closed, X means status unknown, 0 means no DPD, 1 means maximal did during month between 1-30, 2 means DPD 31-60,?5 means DPD 120+ or sold or written off ] )'}],
 [{'bureau.CREDIT_DAY_OVERDUE': 'Number of days past due on CB credit at the time of application for related loan in our sample'}],
 [{'installments_payments.NUM_INSTALMENT_NUMBER': 'On which installment we observe payment'}],
 [{'application_test.NAME_HOUSING_TYPE': 'What is the housing situation of the client (renting, living with parents, ...)'},
  {'appli

In [4]:
# 粗翻译,翻译到接近自然语言,没有特殊标记符号,和类似函数的表达形式.

def rough_tranlate_clause(sentence):
    prompt = """You are a Translator."""

    usercontent = f""" 
    Translate the sentence provided into spoken English, not contain any special symbols except commas and periods.
    Only returns the translated sentence, do not generate any other content, such as "The caluse is: ...".
    if you have trouble doing it or hold that nothing need to be changed, just return the original sentence,
    the returned sentence is either the Translated sentence or the original sentence, but we encourage you to translate the sentence even if the changes are minor.
    The sentence is: 
    {sentence}
    """

    return_format = '{"sentence":"translated sentence"}'

    result = makecomversation(prompt, usercontent, return_format)

    return result['sentence']

def rough_tranlate(clause_list):
    answer = []
    for clause in clause_list:
        sentence = rough_tranlate_clause(clause)
        answer.append(sentence)
    print(answer)
    return answer

# simplified_sqls = []
# for item in decomposed_salika_clauses:
#     simplified_sql = rough_tranlate(item)
#     simplified_sqls.append(simplified_sql)



[{'role': 'system', 'content': 'You are a Translator.'}, {'role': 'user', 'content': ' \n    Translate the sentence provided into spoken English, not contain any special symbols except commas and periods.\n    Only returns the translated sentence, do not generate any other content, such as "The caluse is: ...".\n    if you have trouble doing it or hold that nothing need to be changed, just return the original sentence,\n    the returned sentence is either the Translated sentence or the original sentence, but we encourage you to translate the sentence even if the changes are minor.\n    The sentence is: \n    1 SELECT0  employees.birth_time, salaries.to_time \n     Return in JSON format: {"sentence":"translated sentence"}'}]
[{'role': 'system', 'content': 'You are a Translator.'}, {'role': 'user', 'content': ' \n    Translate the sentence provided into spoken English, not contain any special symbols except commas and periods.\n    Only returns the translated sentence, do not generate an

In [23]:
simplified_sqls

[['SELECT credit_card_balance.AMT_RECEIVABLE_PRINCIPAL',
  'WHERE credit_card_balance.SK_ID_PREV NOT IN (previous.SK_ID_PREV)'],
 ['SELECT installments_payments.SK_ID_PREV',
  'WHERE installments_payments.SK_ID_CURR NOT IN (previous.SK_ID_CURR)'],
 ['SELECT bureau_balance.STATUS',
  'WHERE bureau_balance.SK_ID_BUREAU NOT IN (bureau.SK_ID_BUREAU'],
 ['SELECT pos_cash_balance.SK_DPD',
  'WHERE pos_cash_balance.SK_ID_CURR NOT IN (previous.SK_ID_CURR)'],
 ['SELECT bureau.CREDIT_DAY_OVERDUE',
  'WHERE bureau.SK_ID_BUREAU NOT IN (bureau_balance.SK_ID_BUREAU'],
 ['SELECT installments_payments.NUM_INSTALMENT_NUMBER',
  'WHERE installments_payments.SK_ID_PREV NOT IN (previous.SK_ID_PREV)'],
 ['SELECT application_test.NAME_HOUSING_TYPE',
  'WHERE application_test.AMT_ANNUITY NOT IN (bureau.AMT_ANNUITY'],
 ['Select the count of future installment counts from the POS cash balance table.',
  'Where pos_cash_balance.SK_ID_PREV not in (previous.SK_ID_PREV'],
 ['Select count of SK_DPD_DEF from pos_cas

In [5]:

# few shot Translation
def fewshot_tranlate_clause(clause):
    
    clause_type = clause.split()[0].lower()
    examples = get_examples(clause_type)
    print(clause,examples)

    example_prompt = "Here is some examples:" if len(examples)>0 else ""
    examples = "" if len(examples) == 0 else examples
    prompt = f"""You are a language expert. You need to translate the sql clause provided and then return the colloquial result. 
    {example_prompt}
    {examples}
    """

    usercontent = f""" You need to translate the sql clause provided and then return the colloquial result. 
    Do not generate any other content, such as "The paragraph means: ", just the result. 
    if you can't translate the sql clause, return the clause provided, don't reply content as such"'The SQL clause is not valid. Please provide a valid SQL clause.'".
    The sql clause is: 
    {clause}
    """

    return_format = '{"colloquialized":"colloquialized result"}'

    result = makecomversation(prompt, usercontent, return_format)

    return result['colloquialized']

import random
def get_examples(type):
    with open('Translation_examples.txt', 'r') as f:
        Translation_examples = f.readlines()
    examples = []
    if type == 'select':
        select_examples = list(set([line for line in Translation_examples if line.startswith('select')] ))
        random_selection = random.sample(select_examples, 20)
    elif type == 'where':
        where_examples = list(set([line for line in Translation_examples if line.startswith('where')] ))
        random_selection = random.sample(where_examples, 20)
    elif type == 'order':
        order_examples = list(set([line for line in Translation_examples if line.startswith('order')] ))
        random_selection = random.sample(order_examples, 20)
    elif type == 'group':
        order_examples = list(set([line for line in Translation_examples if line.startswith('group')] ))
        random_selection = random.sample(order_examples, 5)
    else: 
        return []
    

    for line in random_selection:
        parts = line.split('|')
        examples.append({'before': parts[0].strip(), 'colloquialized': parts[1].strip()})
    return examples


def get_hint(simplified_sql):
    sub_questions = []
    for sentence in simplified_sql:
        sub_question = fewshot_tranlate_clause(sentence)
        sub_questions.append(sub_question)
    return sub_questions


# hints = []
# for item in decomposed_salika_clauses:
#     hint = get_hint(item)
#     hints.append(hint)



1 SELECT employees.birth_time, salaries.to_time []
[{'role': 'system', 'content': 'You are a language expert. You need to translate the sql clause provided and then return the colloquial result. \n    \n    \n    '}, {'role': 'user', 'content': ' You need to translate the sql clause provided and then return the colloquial result. \n    Do not generate any other content, such as "The paragraph means: ", just the result. \n    if you can\'t translate the sql clause, return the clause provided, don\'t reply content as such"\'The SQL clause is not valid. Please provide a valid SQL clause.\'".\n    The sql clause is: \n    1 SELECT employees.birth_time, salaries.to_time\n     Return in JSON format: {"colloquialized":"colloquialized result"}'}]
WHERE dept_emp.dept_no equals 'd003' [{'before': 'where first name equal to steve # 2', 'colloquialized': 'steve'}, {'before': 'where hardware model name equal to lg-p760 # 3', 'colloquialized': 'hardware model named `` lg-p760'}, {'before': 'where po

In [19]:

# 最终翻译
def the_final_countdown(original_sql, simplified_version, hints, column_descriptions):
    with open('spider_examples.txt', 'r') as f:
        spider_examples = f.readlines()
    random_selection = random.sample(spider_examples, 10)
    
    prompt = f"""You are a sql to question translation expert. You need to translate the sql provided and then return the coresponding question easy to understand. 
    Especially, when count(stuff_id) or group by stuff_id, we means how many stuff are there , or group by stuff the the ids, not how many ids are there. 
    and don't use the word "count" or orther function name in the question, say what they do.
    Also, don't mention the table name or column name in the question, or use the word "table" or "column" in the question, guess the realistic meaning from the table name, 
    such as table "address.address" is just a address, the "address.last_update" is just the last update time.
    don't use from clause in the question, and don't use the word "from" in the question, people who ask the question don't need to know where the data is stored.
    most sql i provided contains meaningless parts, if possible, try to ignore those meaningless part in the question.
    here is some examples:
    {random_selection}
    """

    usercontent = f""" You need to translate the sql provided and then return coresponding question. 
    Because sometimes SQL is too complex to understand, or even contains errors, we will also provide a simplified version(a clause list, contains main information about the sql) for reference. 
    In addition, we will also provide some hints to introduce what the question will be about.
    lastly, we will provide the descriptions of the columns in the sql, you should use the true meaning in your translation. 
    Do not generate any other content, such as "The question is: ", just the question of the sql itself. 
    if you can't translate the sql, return the simplified version provided(needs to be concat in right order), don't reply content as such "The SQL clause is not valid. Please provide a valid SQL clause.".
    The original sql is: 
    {original_sql}
    The simplified version is:
    {simplified_version}
    The hints are:
    {hints}
    The descriptions of the columns are:
    {column_descriptions}
    """

    return_format = '{"question":" coresponding question to the sql provided"}'

    result = makecomversation(prompt, usercontent, return_format)

    return result['question']

# original_sqls = []
# for item in salika:
#     sql = item['query']
#     original_sqls.append(sql)

# new_salika = []
# for original_sql, simplified_sql, hint, column_descriptions in zip(original_sqls, simplified_sqls, hints, descriptions_list):
#     question = the_final_countdown(original_sql, simplified_sql, hint, column_descriptions)
#     print(question)
#     print('\n\n')

#     new_salika.append({'sql': original_sql, 'question': question})

# new_salika

sql = "SELECT DISTINCT T1.aircraft_code, count(DISTINCT  T1.range) FROM aircrafts_data AS T1  GROUP BY T1.aircraft_code"
clause = preprocess(sql)

clause_list = get_clause(clause)
clause_list = after_process(clause_list)
simplified_sql = rough_tranlate(clause_list)
hint = get_hint(simplified_sql)


column_descriptions = [
       "column aircrafts_data.range means the maximum distance an aircraft can fly between takeoff and landing",
    ]

question = the_final_countdown(sql, simplified_sql, hint, column_descriptions)

[{'role': 'system', 'content': 'You are a SQL parsing expert. You need to remove the aliases in sql statement, then return the SQL statement without the aliases. Here is one examples:\n    WITH clause is not considered in this task.\n    before:\n          "SELECT DISTINCT count(anon_1.c_4200) OVER (PARTITION BY anon_1.description) AS c_4528, \n          anon_1.c_141, \n          anon_1.description \n          FROM (\n            SELECT film_text.description AS description, \n            count(distinct(film_text.film_id)) AS c_141, \n            count(film_text.title) OVER (ORDER BY film_text.title) AS c_4200 \n            FROM film_text \n            WHERE (film_text.film_id >= %(film_id_1)s) \n            GROUP BY film_text.title, film_text.description, film_text.film_id) AS anon_1\n    after: \n          SELECT DISTINCT COUNT(film_text.title) OVER (PARTITION BY film_text.description), \n                COUNT(DISTINCT film_text.film_id) \n                film_text.description\n      

In [20]:
question

'How many different maximum flight distances are there for each unique aircraft code?'

In [27]:
new_salika

[{'sql': 'SELECT T1.AMT_RECEIVABLE_PRINCIPAL FROM credit_card_balance AS T1 WHERE T1.SK_ID_PREV NOT IN (SELECT T22.SK_ID_PREV FROM previous AS T22)',
  'question': 'What are the amounts receivable for principal on credit card balances that are not listed in previous records?'},
 {'sql': 'SELECT T1.SK_ID_PREV FROM installments_payments AS T1 WHERE T1.SK_ID_CURR NOT IN (SELECT T22.SK_ID_CURR FROM previous AS T22)',
  'question': 'What are the IDs of the previous installment payments that are not associated with any previous credits in our sample?'},
 {'sql': 'SELECT T1.STATUS FROM bureau_balance AS T1 WHERE T1.SK_ID_BUREAU NOT IN (SELECT T22.SK_ID_BUREAU FROM bureau AS T22)',
  'question': 'What are the statuses of bureau balance records that do not have a corresponding bureau ID?'},
 {'sql': 'SELECT T1.SK_DPD FROM pos_cash_balance AS T1 WHERE T1.SK_ID_CURR NOT IN (SELECT T22.SK_ID_CURR FROM previous AS T22)',
  'question': 'What are the days past due for accounts in the POS cash balance

In [28]:
with open('column_indicator_test.json', 'w', encoding='utf-8') as f:
    json.dump(new_salika, f, ensure_ascii=False)

In [1]:
from src.qa import makecomversation

"""
方法测试，

1. 并行性也就是经济性

2. token统计

3. 有效性，尤其在返回格式上

如果能够确保并行下的有效性，那么下一步需要确定是否采用异步的方法，

异步方法，可以提高效率和经济性。整个过程可以分成三个部分，第一部分是预处理，第二部分是知识生成，第三部分是翻译，第四部分是中文翻译。

"""


def refine(pairs):
    inputs = []
    for index, pair in enumerate(pairs):
        english = pair['en']
        chinese = pair['ch']
        inputs.append(f'pair {index}:'+f'english:{english}, its chinese translation:{chinese}')

    prompt = """You are a language expert in English and Chinese."""

    usercontent = f""" 以下的中文翻译有问题吗？稍微改进一下(refine)以使其更自然。
    {inputs}
    """

    return_format = '[{"pair 1":"refined chinese"},{"pair 2":"refined chinese"},...]'

    result, input_tokens, output_tokens = makecomversation(prompt, usercontent, return_format)

    return result, input_tokens, output_tokens

pairs = [{"en": "What are the home states of the shortest players?", "ch": "最矮的球员的家乡州是哪些？"},
        {"en": "What are the hometown cities of the shortest players who are all-stars?", "ch": "哪些是全明星中最矮球员的家乡城市？"}]


result, input_tokens, output_tokens = refine(pairs)
print(input_tokens, output_tokens)
result

169 58


{'pair 0': '最矮球员的家乡所在州是哪些？', 'pair 1': '全明星中最矮的球员的家乡城市有哪些？'}